In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import Dataset

class AudioDataset(Dataset):
    def __init__(self, data, labels, transform=None):
        self.data = data
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample, self.labels[idx]


dataset = torch.load('dataset.pth')
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])
X_train, y_train = train_dataset.dataset[torch.tensor(train_dataset.indices)]
X_test, y_test = test_dataset.dataset[torch.tensor(test_dataset.indices)]



# Erstellen des CNN-Modells
print("defining model")
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 32, (3, 3))
        self.pool1 = nn.MaxPool1d((2, 2))
        self.conv2 = nn.Conv1d(32, 64, (3, 3))
        self.pool2 = nn.MaxPool1d((2, 2))
        self.fc1 = nn.Linear(64 * 4 * 4, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = self.pool1(F.relu(self.conv1(x)))
        x = self.pool2(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 4 * 4)
        x = F.relu(self.fc1(x))
        x = torch.sigmoid(self.fc2(x))
        return x

# Initialisieren des Modells
model = CNN()

# Definieren des Optimierers und der Verlustfunktion
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

TypeError: only integer tensors of a single element can be converted to an index

In [ ]:
print("training model")
# Trainieren des Modells
for epoch in range(10):
    # Trainieren des Modells mit einem Batch
    outputs = model(X_train)
    loss = criterion(outputs, y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Drucken des Verlusts
    print(f"Epoch {epoch + 1}: {loss.item()}")


# # Bewertung des Modells
# with torch.no_grad():
#     outputs = model(X_test)
#     _, predicted = torch.max(outputs.data, 1)
#     accuracy = (predicted == y_test).sum().item() / len(y_test)

# print(f"Accuracy: {accuracy}")

# Speichern des Modells
torch.save(model.state_dict(), "model.pt")
